[Reference](https://code.likeagirl.io/how-indexing-speeds-up-lookups-a-technical-deep-dive-0fe64b98b4d8)

In [1]:
import time
from sqlalchemy import create_engine, Column, Integer, String
from sqlalchemy.orm import sessionmaker

# Create tables with and without an index
engine = create_engine('sqlite:///example.db')
Base = declarative_base()

class Book(Base):
    __tablename__ = 'books'
    id = Column(Integer, primary_key=True)
    title = Column(String)
Base.metadata.create_all(engine)

# Generate sample data
session = sessionmaker(bind=engine)()
for i in range(10000):
    session.add(Book(title=f'Book {i}'))
session.commit()
# Measure query times with and without indexing

def query_with_index():
    start_time = time.time()
    books = session.query(Book).filter(Book.title == 'Book 9999').all()
    end_time = time.time()
    print(f"Index scan time: {end_time - start_time:.3f} seconds")

def query_without_index():
    start_time = time.time()
    books = session.query(Book).filter(Book.title == 'Book 9999').all()
    end_time = time.time()
    print(f"Sequential scan time: {end_time - start_time:.3f} seconds")

query_without_index()

# Create an index on the title column
session.execute('CREATE INDEX idx_title ON books (title)')

# Run the queries
query_with_index()
# Close the session
session.close()